In [ ]:
!pip install transformers
!pip install peft
!pip install 'accelerate>=0.26.0'
!pip install -U bitsandbytes
!pip install huggingface-hub
!pip install datasets
!pip install wandb

In [1]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          Trainer,
                          pipeline,
                          DataCollatorForLanguageModeling,
                          PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model
import huggingface_hub
import os
import logging
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from typing import List
import wandb
from lora_llm import finetuning, evaluate, inference

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'lora_llm'
os.environ["WANDB_DISABLED"] = "false"

# Load the API key from the secret.json file
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    huggingface_hub.login(secrets.get('HF_KEY'))
    wandb.login(key=secrets.get('WANDB_KEY'))

lr = 2e-4
epochs = 2
class_names = []
base_model_name = 'google/gemma-2-9b-it'
label_name = "evasion_label"
fine_tuned_model_path = f"./gemma_9b_{epochs}ep"

# Wandb configuration
run = wandb.init(entity="kontilenia-national-technical-university-of-athens",
                 project='political-speech-clarity',
                 job_type="training",
                 name=str(epochs)+" epoch Gemma",
                 # Track hyperparameters and run metadata
                 config={
                    "learning_rate": lr,
                    "architecture": base_model_name,
                    "dataset": "qevasion_dataset_preproccessed",
                    "epochs": epochs,
                 })

model, tokenizer = finetuning(base_model_name,
                              fine_tuned_model_path,
                              label_name,
                              lr,
                              epochs)

evaluate(base_model_name,
         fine_tuned_model_path,
         "evasion_label",
         "clarity_label",
         "preprocessed_data/test_set.csv",
         False,
         model,
         tokenizer,
         run)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find lora_llm.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: Currently logged in as: kontilenia (kontilenia-national-technical-university-of-athens) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 8945664 || all params: 5088853504
        || trainable%: 0.17578937953251011
Found 3109 instances for training and
    339 instances for validation.
Training...


Step,Training Loss,Validation Loss
129,1.810300,1.801431
258,1.916900,1.773488
387,1.734200,1.744111
516,1.750900,1.754110
645,1.615500,1.725847
774,1.624000,1.718775


...

 97%|█████████▋| 34/35 [03:11<00:05,  5.61s/it]

Right label: explicit
Right label: explicit
Right label: dodging
Right label: general
Right label: dodging
Right label: dodging
Right label: deflection
Right label: implicit


100%|██████████| 35/35 [03:14<00:00,  5.55s/it]

Right label: implicit
Right label: deflection
Right label: general
Right label: general
Accuracy: 0.70
Accuracy for label indirect: 0.95
Accuracy for label direct reply: 0.34
Accuracy for label direct non-reply: 0.09

Classification Report:
                  precision    recall  f1-score   support

        indirect       0.70      0.95      0.81       174
    direct_reply       0.73      0.34      0.47        79
direct_non-reply       1.00      0.09      0.16        23

       micro avg       0.71      0.70      0.71       276
       macro avg       0.81      0.46      0.48       276
    weighted avg       0.73      0.70      0.66       276


Confusion Matrix:
[[165   9   0]
 [ 52  27   0]
 [ 18   1   2]]


['Accuracy: 0.70',
 'Accuracy for label indirect: 0.95',
 'Accuracy for label direct reply: 0.34',
 'Accuracy for label direct non-reply: 0.09',
 'Classification Report:                  precision    recall  f1-score   support\n\n        indirect       0.70      0.95      0.81       174\n    direct_reply       0.73      0.34      0.47        79\ndirect_non-reply       1.00      0.09      0.16        23\n\n       micro avg       0.71      0.70      0.71       276\n       macro avg       0.81      0.46      0.48       276\n    weighted avg       0.73      0.70      0.66       276\n',
 'Confusion Matrix:[[165   9   0]\n [ 52  27   0]\n [ 18   1   2]]']

In [2]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'lora_llm'
os.environ["WANDB_DISABLED"] = "false"

# Load the API key from the secret.json file
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    huggingface_hub.login(secrets.get('HF_KEY'))
    wandb.login(key=secrets.get('WANDB_KEY'))

lr = 2e-4
epochs = 2
class_names = []
base_model_name = 'google/gemma-2-27b-it'
label_name = "evasion_label"
fine_tuned_model_path = f"./gemma_9b_{epochs}ep"

evaluate(base_model_name,
         fine_tuned_model_path,
         "evasion_label",
         "clarity_label",
         "preprocessed_data/named_test_set_info.csv",
         added_name_summary=False)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find lora_llm.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: Currently logged in as: kontilenia (kontilenia-national-technical-university-of-athens) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
  8%|▊         | 1/13 [00:05<01:09,  5.78s/it]

Right label: implicit
Right label: implicit
Right label: dodging
Right label: dodging
Right label: implicit
Right label: implicit
Right label: implicit
Right label: implicit


 15%|█▌        | 2/13 [00:12<01:06,  6.06s/it]

...

Right label: implicit
Right label: implicit
Right label: implicit
Right label: implicit
Right label: general
Right label: implicit
Right label: implicit
Right label: implicit


100%|██████████| 13/13 [01:09<00:00,  5.32s/it]

Right label: implicit
Accuracy: 0.76
Accuracy for label indirect: 0.95
Accuracy for label direct reply: 0.43
Accuracy for label direct non-reply: 0.00

Classification Report:
                  precision    recall  f1-score   support

        indirect       0.75      0.95      0.84        64
    direct_reply       0.87      0.43      0.58        30
direct_non-reply       0.00      0.00      0.00         3

       micro avg       0.77      0.76      0.77        97
       macro avg       0.54      0.46      0.47        97
    weighted avg       0.76      0.76      0.73        97


Confusion Matrix:
[[61  2  0]
 [17 13  0]
 [ 3  0  0]]



/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

['Accuracy: 0.76',
 'Accuracy for label indirect: 0.95',
 'Accuracy for label direct reply: 0.43',
 'Accuracy for label direct non-reply: 0.00',
 'Classification Report:                  precision    recall  f1-score   support\n\n        indirect       0.75      0.95      0.84        64\n    direct_reply       0.87      0.43      0.58        30\ndirect_non-reply       0.00      0.00      0.00         3\n\n       micro avg       0.77      0.76      0.77        97\n       macro avg       0.54      0.46      0.47        97\n    weighted avg       0.76      0.76      0.73        97\n',
 'Confusion Matrix:[[61  2  0]\n [17 13  0]\n [ 3  0  0]]']